In [158]:
#python3 t2_s2_tile_nob.py --fromf train_size2 --tof ts2_16x320_inv_nob --ntile 16 --tsize 80

In [159]:
#makes simple tile from train_size1 images

In [1]:
DEBUG = True

In [2]:
import os, gc, subprocess, time, sys, shutil, argparse

import scipy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm
#from tqdm.notebook import tqdm
import skimage.io
from skimage.transform import resize, rescale
from math import sqrt
import statistics

sys.path.append("..")

In [3]:
import PIL.Image
#PIL.Image.MAX_IMAGE_PIXELS = 400000000
PIL.Image.MAX_IMAGE_PIXELS = None

In [4]:
from panda_bvv_config import *

## 1. Inputs ##

In [5]:
Zoom = 2
#input_folder = train_size2
input_mask_folder = mask_size2


In [6]:
if DEBUG:
    input_folder_name = 'cancer_s2'
    to_folder = 'prob_folder'
    ntile = 16
    tsize = 80
else:
    ap = argparse.ArgumentParser()
    ap.add_argument('--fromf', help="folder name to grabb files fot tiling", dest = 'fromf',\
                    type=str)
    ap.add_argument('--tof', help="folder name to save tile", dest = 'tof',\
                    type=str)
    ap.add_argument('--ntile', help="number of tiles", dest = 'ntile', default = 16,\
                    type=int)
    ap.add_argument('--tsize', help="size of one square tile", dest = 'tsize', default = 80,\
                    type=int)# default for B3 net

                    #("--cnn", type=str, help="training cnn name", dest = 'train_cnn')
    args = vars(ap.parse_args())
    input_folder_name = args["fromf"]
    to_folder = args["tof"]
    ntile = args["ntile"]
    tsize = args["tsize"]

In [7]:
#data-from folder:


In [8]:
kw_for_tile_func = {'sz': tsize, 'N':ntile, 'threshold':241, 'porog':4700000,}
new_proc_folder = os.path.join(base_path, to_folder)
input_folder = os.path.join(base_path, input_folder_name)

In [9]:
# last_train_label = 9000
# last_valid_label = 10617
#kw = {'gauss_blur':20}

In [10]:
# wrapping inputs:
if not os.path.exists(new_proc_folder):
    print("[INFO] 'creating {}' directory".format(new_proc_folder))
    os.makedirs(new_proc_folder)


## 2. Standard functions ##

In [11]:
proc_dict = {'input_biopsy_folder':input_folder,
            'output_data_path':new_proc_folder,
            'df_name':test_cnn_labels,
}


In [12]:
proc_dict = {**proc_dict, **kw_for_tile_func }
#proc_dict

In [13]:
#wrapper for image processing function
def process_all_images(proc_func):
    
    def wrapper(proc_dict):
        zoom = Zoom
        input_biopsy_folder = proc_dict.get('input_biopsy_folder')
        output_data_path = proc_dict.get('output_data_path')
        df_name = proc_dict.get('df_name')
        tif_file = False
        with_mask = False
        input_mask_folder = mask_size2
        
        print(input_biopsy_folder)
        print(new_proc_folder)
        with tqdm(total=df_name.shape[0]) as pbar:
            for  i, row in enumerate(tqdm(df_name.iterrows())):
                img_num = row[1]['image_id']
                provider = row[1]['data_provider']                         
                mask = None
                if tif_file:
                    try:
                        biopsy = skimage.io.MultiImage(os.path.join(input_biopsy_folder,                                                        img_num + '.tiff'))[zoom]               
                    except:
                        print('Failed to read tiff:', img_num)                        
                else:
                    try:
                        biopsy = skimage.io.imread(os.path.join(input_biopsy_folder, img_num + '.png'))
                        shape = biopsy.shape
                        if shape[0]<shape[1]: 
                            biopsy = np.rot90(biopsy)
                            shape = biopsy.shape 
                    except:
                        print(f'can not proceed with {img_num}')   
                if with_mask:
                    try:
                        mask = skimage.io.imread(os.path.join(input_mask_folder, img_num + '_mask.png'))
                        shape = mask.shape
                        if shape[0]<shape[1]: 
                            mask = np.rot90(mask)
                            shape = mask.shape 
                    except:
                        print('Failed to process mask:', img_num)
                try:
                    data_new = proc_func(biopsy, **proc_dict)
                    if data_new.any(): cv2.imwrite(os.path.join(output_data_path, img_num + '.png'), data_new)
                except Exception as ee:
                    print('Processing mistake:\n', ee, '\n', img_num)            
                try:
                    del biopsy, mask, data_new, mask_new
                except:
                    pass
                pbar.update(1)
            gc.collect()
        return
    
    return wrapper

## 3. Processing functions ##

In [31]:
#@process_all_images
def tile16_pad(img, **kw):        
    
    threshold = kw['threshold']
    porog = kw['porog']
    sz=kw.get('sz') if kw.get('sz') else 32    
    N = kw.get('N') if kw.get('N') else 16
    final_dim = kw.get('final_dim') if kw.get('final_dim') else N**0.5 
    grey_biopsy = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, white_mask = cv2.threshold(grey_biopsy, threshold, 255, 0)
    black_mask = cv2.bitwise_not(white_mask)
    img[black_mask==0] = (255,255,255)
    
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    
    try:
        yy = [y for y in idxs if img.reshape(img.shape[0],-1).sum(-1)[y] > porog] #4777000
        ee = idxs[:N-len(yy)]
        eee = np.pad(ee, [0,len(yy)], 'wrap' )
        np.random.shuffle(eee)
        assert len(ee)!=0, print('Dlina 0')
        img = img[eee]
    except:
        print('Ne srabotalo')
        return None
    a, b, c, d = img.shape
    
    data_new = (img.reshape(int(final_dim), int(final_dim),\
                                                    b, c, d).swapaxes(1,2).reshape(b*int(final_dim),\
                                                    c*int(final_dim), d))
    data_new = cv2.bitwise_not(data_new)
    #skimage.io.imshow(data_new)
    return data_new

In [15]:
#tt = tile16_simple(img, mask, provider, **kw)

## 4. Process input files for training purpose ##

In [16]:
%time
tile16_pad(proc_dict)

  0%|          | 0/1216 [00:00<?, ?it/s]
0it [00:00, ?it/s]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
/run/media/admin/kagg/panda/cancer_s2
/run/media/admin/kagg/panda/prob_folder


  0%|          | 5/1216 [00:01<06:52,  2.94it/s]
5it [00:01,  2.94it/s]

Ne srabotalo
Processing mistake:
 'NoneType' object has no attribute 'any' 
 66f04788a17915feba394290384e13a4


  3%|▎         | 35/1216 [00:11<06:03,  3.25it/s]
35it [00:11,  3.25it/s]

...

 93%|█████████▎| 1135/1216 [06:21<00:22,  3.68it/s]
1135it [06:21,  3.68it/s]

can not proceed with a109197ed889dd998c52e3f9d0bd98f0
Processing mistake:
 local variable 'biopsy' referenced before assignment 
 a109197ed889dd998c52e3f9d0bd98f0


 99%|█████████▉| 1201/1216 [06:44<00:06,  2.44it/s]
1201it [06:44,  2.44it/s]

can not proceed with 6df7a3d8b4955d162fcc11bce003860a
Processing mistake:
 local variable 'biopsy' referenced before assignment 
 6df7a3d8b4955d162fcc11bce003860a


100%|██████████| 1216/1216 [06:48<00:00,  3.10it/s]
1216it [06:48,  2.98it/s]
100%|██████████| 1216/1216 [06:48<00:00,  2.98it/s]


In [17]:
from multiprocessing import Pool
from multiprocessing import cpu_count

In [18]:
procs = cpu_count()

In [19]:
def chunk(l, n):
    # loop over the list in n-sized chunks
    for i in range(0, len(l), n):
        # yield the current n-sized chunk to the calling function
        yield l[i: i + n]

In [20]:
next(chunk(os.listdir(input_folder), 4))

['d3a5775345ddc4976944ff1941709ef0.png',
 'e20c73041c5c5ac83c20a7c669f46e9a.png',
 '8f8d20fc6e9b6aa3b145e359e104eac4.png',
 '6ac2095c7968fa30ab6c5dc7d1ed5121.png']

In [21]:
def chunk_df(df, n):
    # loop over the list in n-sized chunks
    for i in range(0, df.shape[0], n):
        # yield the current n-sized chunk to the calling function
        yield df[i: i + n]

In [22]:
next(chunk_df(test_cnn_labels, 4)).shape

(4, 5)

In [44]:
allImagePaths = sorted(os.listdir(input_folder))
numImagesPerProc = len(allImagePaths) / float(procs)
numImagesPerProc = int(np.ceil(numImagesPerProc))

In [42]:
len(allImagePaths)

1201

In [37]:
numImagesPerProc 

301

In [38]:
chunkedPaths = list(chunk(allImagePaths, numImagesPerProc))

In [39]:
len(chunkedPaths)

4

In [43]:
test_cnn_labels.shape

(1216, 5)

In [46]:
allImagedf = test_cnn_labels.shape[0]
numdfPerProc = allImagedf / float(procs)
numdfPerProc = int(np.ceil(numdfPerProc))

In [47]:
chunked_df = list(chunk_df(test_cnn_labels, numdfPerProc))

In [48]:
len(chunked_df)

4

In [49]:
proc_dict = {'input_biopsy_folder':input_folder,
            'output_data_path':new_proc_folder,
}
kw_for_tile_func = {'sz': tsize, 'N':ntile, 'threshold':241, 'porog':4700000,}

In [50]:
payloads = []
for i, dff in enumerate(chunked_df):
    data_temp = {
        'df_name':dff,
        
    }
    payloads.append({**data_temp, **proc_dict, **kw_for_tile_func})


In [51]:
len(payloads)

4

In [64]:
pool = Pool(processes = procs)

/run/media/admin/kagg/panda/cancer_s2
/run/media/admin/kagg/panda/prob_folder
/run/media/admin/kagg/panda/cancer_s2
/run/media/admin/kagg/panda/cancer_s2
/run/media/admin/kagg/panda/cancer_s2
/run/media/admin/kagg/panda/prob_folder
/run/media/admin/kagg/panda/prob_folder
/run/media/admin/kagg/panda/prob_folder


  0%|          | 0/304 [00:00<?, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  1%|          | 3/304 [00:01<02:22,  2.11it/s]
3it [00:01,  2.17it/s]
  2%|▏         | 5/304 [00:01<02:05,  2.39it/s]
5it [00:01,  2.49it/s]

Ne srabotalo
Processing mistake:
 'NoneType' object has no attribute 'any' 
 66f04788a17915feba394290384e13a4


  7%|▋         | 22/304 [00:10<02:40,  1.76it/s]
22it [00:10,  1.76it/s]

Ne srabotalo
Processing mistake:
 'NoneType' object has no attribute 'any' 
 efd993cb48dce20304958c45829a3ff8


...

 94%|█████████▍| 285/304 [01:57<00:06,  2.86it/s]

 6df7a3d8b4955d162fcc11bce003860a



 96%|█████████▌| 291/304 [01:59<00:04,  2.74it/s]
291it [01:59,  2.75it/s]

can not proceed with c3ca4a75be0055aefe79f6849804d62c
Processing mistake:
 local variable 'biopsy' referenced before assignment 
 c3ca4a75be0055aefe79f6849804d62c


100%|██████████| 304/304 [02:01<00:00,  3.08it/s]
304it [02:01,  2.50it/s]
100%|██████████| 304/304 [02:03<00:00,  3.41it/s]
304it [02:03,  2.46it/s]
100%|██████████| 304/304 [02:05<00:00,  2.92it/s]
304it [02:05,  2.42it/s]
100%|██████████| 304/304 [02:05<00:00,  2.42it/s]


In [55]:
def new_tile(*args, **kw):
    return process_all_images(tile16_pad)(*args, *kw)

In [56]:
%%time
pool.map(new_tile, payloads)
print("[INFO] waiting for processes to finish...")
pool.close()
pool.join()
print("[INFO] multiprocessing complete")


[INFO] waiting for processes to finish...
[INFO] multiprocessing complete
CPU times: user 3.67 s, sys: 1.33 s, total: 5 s
Wall time: 2min 9s


In [65]:
%%time
pool.map_async(new_tile, payloads)
print("[INFO] waiting for processes to finish...")
pool.close()
pool.join()
print("[INFO] multiprocessing complete")

[INFO] waiting for processes to finish...
[INFO] multiprocessing complete
CPU times: user 3.56 s, sys: 1.37 s, total: 4.93 s
Wall time: 2min 5s


In [169]:
import os
module_name = 't2_s2_tile_nob'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)